# Install protinfer 

Note protinfer was not re-trained so may be slightly biased.

Protinfer needs to be installed as per the guidelines and then the following command (then CARE will take care of the rest!)

```
git clone https://github.com/google-research/proteinfer
cd ~/proteinfer
pip3 install -r requirements.txt
python3 install_models.py
```

Example to run (you don't need this for our benchmarking pipeline but it may be useful to know!)

```
python3 proteinfer.py -i price.fasta -o price.tsv
```


## Run CARE

In [1]:
import sys
sys.path.append('/disk1/ariane/vscode/CARE/')
sys.path.append('/disk1/ariane/vscode/CARE/CARE/')

from CARE.processing import *
from CARE.task1 import *

data_dir = '/disk1/ariane/vscode/CARE/data/task1/'
output_dir = '/disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/'

In [2]:
# Save in the required format
# Make a task1
tasker = Task1(data_folder=data_dir, output_folder=output_dir)
for split in ['30', '30-50', 'price', 'promiscuous']:
    # For proteInfer you need to point where it was saved.
    tasker.get_proteinfer(split, proteinfer_dir='/disk1/ariane/pycharm/CARE/proteinfer/', save=True)

--------------------------------------------------------------------------------
        Warning! You must already have the proteInfer environment made	         
--------------------------------------------------------------------------------
/disk1/ariane/vscode/CARE/data/task1/30_protein_test.fasta





I0604 10:51:22.242385 139967170348864 proteinfer.py:392] Running with 1 ensemble elements
I0604 10:51:22.242534 139967170348864 proteinfer.py:189] Parsing input from /disk1/ariane/vscode/CARE/data/task1/30_protein_test.fasta
Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.30it/s]

Annotating batches of sequences: 100%|██████████| 3/3 [00:06<00:00,  2.09s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:05<00:00,  1.94s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:05<00:00,  1.96s/it]]

Progress: 100%|██████████| 3/3 [00:18<00:00,  6.07s/it]
I0604 10:51:42.862294 139967170348864 proteinfer.py:340] Writing output to /disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/30_proteInfer.tsv



--------------------------------------------------------------------------------
Done: 	30	
Saved to:	/disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/30_protein_test_results_df.csv	
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
        Warning! You must already have the proteInfer environment made	         
--------------------------------------------------------------------------------
/disk1/ariane/vscode/CARE/data/task1/30-50_protein_test.fasta





I0604 10:51:45.946799 140480805988160 proteinfer.py:392] Running with 1 ensemble elements
I0604 10:51:45.946943 140480805988160 proteinfer.py:189] Parsing input from /disk1/ariane/vscode/CARE/data/task1/30-50_protein_test.fasta
Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.29it/s]

Annotating batches of sequences: 100%|██████████| 4/4 [00:06<00:00,  1.72s/it]

Annotating batches of sequences: 100%|██████████| 4/4 [00:06<00:00,  1.53s/it]

Annotating batches of sequences: 100%|██████████| 4/4 [00:06<00:00,  1.66s/it]

Progress: 100%|██████████| 3/3 [00:19<00:00,  6.61s/it]
I0604 10:52:08.215528 140480805988160 proteinfer.py:340] Writing output to /disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/30-50_proteInfer.tsv



--------------------------------------------------------------------------------
Done: 	30-50	
Saved to:	/disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/30-50_protein_test_results_df.csv	
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
        Warning! You must already have the proteInfer environment made	         
--------------------------------------------------------------------------------
/disk1/ariane/vscode/CARE/data/task1/price_protein_test.fasta



--------------------------------------------------------------------------------
Done: 	price	
Saved to:	/disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/price_protein_test_results_df.csv	
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
        Warning! You must alrea

I0604 10:52:11.314195 139874811864896 proteinfer.py:392] Running with 1 ensemble elements
I0604 10:52:11.314341 139874811864896 proteinfer.py:189] Parsing input from /disk1/ariane/vscode/CARE/data/task1/price_protein_test.fasta
Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.31it/s]

Annotating batches of sequences: 100%|██████████| 3/3 [00:05<00:00,  1.79s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:05<00:00,  1.67s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:05<00:00,  1.72s/it]]

Progress: 100%|██████████| 3/3 [00:15<00:00,  5.25s/it]
I0604 10:52:29.446339 139874811864896 proteinfer.py:340] Writing output to /disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/price_proteInfer.tsv



## Train the model
Future work may include training the model:
```
python train.py --data_base_path=./testdata/ \
--label_vocab_path=./data/vocabs/EC.tsv \
--hparams_set=small_test_model \
--output_dir=trained_baseline1
```